In [1]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta

In [2]:
# path to your service account key JSON file
key_file_path = "./smartwaiver-388021-9a8ea3b56852.json"

# Use your JSON key to authenticate your account
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name(key_file_path, scope)

client = gspread.authorize(creds)


In [3]:
# Add name of goolge spread sheet here
sheet_name = '0713 Ecology Action Check-in'

# Open the Google Spreadsheet by its name (make sure you've shared it with the client_email from your service account)
sheet = client.open(sheet_name).sheet1

In [4]:
# Put the full file name for the smartwaiver data here
smartwaiver = 'smartwaiver-177027-64aef0041ae1a.csv'

# Load the data from your CSV file
data = pd.read_csv(smartwaiver)

In [5]:
# Convert 'Waiver Date' column to datetime
data['Date Completed (UTC)'] = pd.to_datetime(data['Date Completed (UTC)'])

In [6]:
# Get the current date
now = datetime.now()

# Calculate the date one year ago
one_year_ago = now - timedelta(days=365)

In [7]:
# Filter the data to get volunteers who signed a waiver in the past year
filtered_data = data[data['Date Completed (UTC)'] >= one_year_ago]

In [8]:
# View filtered dataframe
filtered_data.head(5)

,First,Middle,Last,Date of Birth,Date Completed (UTC),Waiver ID,Status,Check-ins,Gender,Phone,Requested copy of waiver via email,Email,signing_type,template_type,Title of Document,typed_font
0,Jamie,NaN,Alonzo,NaN,2023-06-26 21:15:17,zLiGrcAXPLRJCJNAsQEVqS,Completed Online,0,NaN,NaN,1,jalonzo@ecoact.org,draw,1,SCMTS Waiver,dancing
1,Mimi,NaN,Martinez,NaN,2023-06-27 20:18:15,9hscHjk5dP74ZMQZh3YVck,Completed Online,0,NaN,NaN,1,nmartinezcsulb@gmail.com,draw,1,SCMTS Waiver,dancing
2,Amelia,NaN,Conlen,NaN,2023-06-27 20:28:24,KPieDx5DJECFHDyqUuuYrK,Completed Online,0,NaN,NaN,1,conlen.ameliawren@gmail.com,draw,1,SCMTS Waiver,dancing
3,Margaret,NaN,Ireland,NaN,2023-06-27 20:35:29,rZPsNtEqALZ2QC9Nop9gpK,Completed Online,0,NaN,NaN,1,margaret.ireland@ecoact.org,draw,1,SCMTS Waiver,dancing
4,Salvador,NaN,Murillo,NaN,2023-06-27 20:47:14,P36GGvYh8BaozkP5wMwAUd,Completed Online,0,NaN,NaN,1,salvador.murillo.g@gmail.com,draw,1,SCMTS Waiver,dancing


In [9]:
# Get all records from the Google Sheet
records = sheet.get_all_records()

# Create an empty DataFrame to store unmatched names
unmatched_names = pd.DataFrame(columns=['First Name', 'Last Name', 'Preferred Email'])

In [10]:

# Get the list of column names
headers = sheet.row_values(1)

# Find the index of the 'SCMTS Waiver' column (Python index starts at 0 so we need to add 1 to match gspread formatting)
waiver_column_index = headers.index('Waiver') + 1

# Iterate over the filtered data
for index, row in filtered_data.iterrows():
    
    # Find the volunteer in the Google Spreadsheet
    matched_records = [record for record in records if record['Contact Name'].split(", ")[1] == row['First'] and record['Contact Name'].split(", ")[0] == row['Last']]

    if matched_records:
          
        # If there is a match check the box
        record = matched_records[0]
        
        row_number = records.index(record) + 2 # Adds 2 because gspread row index starts at 1 and we have a header row

        # Check the box in the 4th column
        sheet.update_cell(row_number, 4, True) # The number 3 reflects the column number

In [11]:
# Iterate over the records from the Google Sheets
for record in records:
    # Check if the box is still 'False'
    if record['Waiver'] == 'FALSE':  # gspread recognizes empty checkboxes as a string 'FALSE'
        # If it's still 'False', add the contact name and email to the DataFrame
        name_parts = record['Contact Name'].split(", ")
        if len(name_parts) == 2: # Making sure the name is in "Last, First" format
            unmatched_names = unmatched_names.append({'First': name_parts[1], 'Last': name_parts[0], 'Email': record['Preferred Email']}, ignore_index=True)

In [12]:
# View dataframe for individuals that did not sign the waiver (to be used for smart_gmail)
unmatched_names

,First Name,Last Name,Preferred Email,Email,First,Last
0,NaN,NaN,NaN,julie.aguilar@ecoact.org,Julieanna,Aguilar
1,NaN,NaN,NaN,jalonzo@ecoact.org,Jamie,Alonzo
2,NaN,NaN,NaN,annalisa.carrillo@ecoact.org,Annalisa,Carrillo-Fulk
3,NaN,NaN,NaN,jrosales@ecoact.org,Jasmine,Castillo Rosales
4,NaN,NaN,NaN,juan.castillo@ecoact.org,Juan,Castillo
5,NaN,NaN,NaN,conlen.ameliawren@gmail.com,Amelia,Conlen
6,NaN,NaN,NaN,sebastien.garbe@ecoact.org,Sebastien,Garbe
7,NaN,NaN,NaN,mirelandm@gmail.com,Margaret,Ireland
8,NaN,NaN,NaN,karina.javier@ecoact.org,Karina A,Javier Casillas
9,NaN,NaN,NaN,tkennedy@ecoact.org,Tawn,Kennedy
